Raul Jimenez - 19017
# Hoja de trabajo Análisis de Malware

## Parte 1 - Analisis estadistico

1. Utilice la herramienta pefile para examinar el PE header y obtenga las DLL y las APIs que cada 
uno  de  los  ejecutables  utilizan.  ¿Qué  diferencias  observa  entre  los  ejemplos?  ¿Existe  algún 
indicio sospechoso en la cantidad de DLLs y las APIs llamadas? 

In [4]:
import pefile

pe_qwrty = pefile.PE('./MALWR/sample_qwrty_dk2')
pe_vg655 = pefile.PE('./MALWR/sample_vg655_25th.exe')

print('SAMPLE_QWRTY_DK2')
for entry in pe_qwrty.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS AL DLL')
    print(entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

print('SAMPLE_VG655_25TH.EXE')
for entry in pe_vg655.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS AL DLL')
    print(entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

Exception: Unable to access file './MALWR/sample_vg655_25th.exe': [Errno 22] Invalid argument: './MALWR/sample_vg655_25th.exe'

2. Obtenga la información de las secciones del PE Header. ¿Qué significa que algunas secciones 
tengan como parte de su nombre “upx”? Realice el procedimiento de desempaquetado para 
obtener las llamadas completas de las APIs. 
3. Según el paper “Towards Understanding Malware Behaviour by the Extraction of API Calls”, 
¿en  que  categoría  sospechosas  pueden  clasificarse  estos  ejemplos  en  base  a  algunas  de  las 
llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de 
malware que el paper propone. 
4. Para el archivo “sample_vg655_25th.exe” obtenga el HASH en base al algoritmo SHA256. 
5. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la DLL ADVAPI32.dll? 
6. Para  el  archivo  “sample_vg655_25th.exe”,  ¿cuál  es  el  propósito  de  la  API 
CryptReleaseContext? 
7. Con  la  información  recopilada  hasta  el  momento,  indique  para  el  archivo 
““sample_vg655_25th.exe” si es sospechoso o no, y cual podría ser su propósito. 

## Part 2 - Analisis dinamico

8. Utilice la plataforma de análisis dinámico https://www.hybrid-analysis.com y cargue el archivo 
“sample_vg655_25th.exe”. ¿Se corresponde el HASH de la plataforma con el generado? ¿Cuál 
es el nombre del malware encontrado? ¿Cuál es el propósito de este malware? 
9. Muestre  las  capturas  de  pantalla  sobre  los  mensajes  que  este  malware  presenta  a  usuario. 
¿Se corresponden las sospechas con el análisis realizado en el punto 7? 